In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
from utils import load_car_data

df = load_car_data(Path("../data/fuel_consumption.csv"))

features = [
    "make",
    "release_year",
    "vehicle_class",
    "fuel_type",
    "transmission_type",
    "gears",
    "engine_size",
    "cylinders",
]
numerical_features = ["release_year", "gears", "engine_size", "cylinders"]
categorical_features = ["make", "vehicle_class", "fuel_type", "transmission_type"]

targets = ["emissions", "fc_mixed", "fc_city", "fc_highway"]

X = df[features]
Y = df[targets]

2025-07-05 18:03:01.869 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-07-05 18:03:01.871 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-07-05 18:03:01.871 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-07-05 18:03:01.871 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-05 18:03:01.956 
  command:

    streamlit run /Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-05 18:03:01.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-05 18:03:01.957 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-05 1

### Model definition

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        (
            "numerical",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                    ("scaler", StandardScaler()),
                ]
            ),
            numerical_features,
        ),
    ]
)

pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", Lasso(max_iter=10000))]
)

param = {"regressor__alpha": np.logspace(-4, 4, 5)}

In [4]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, refit=True)
grid.fit(X, Y)

print(f"Best parameters : {grid.best_params_}")
print(f"Best score : {grid.best_score_:0.2f}")

/Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.080e+05, tolerance: 6.285e+03
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.038e+06, tolerance: 6.262e+03
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to i

Best parameters : {'regressor__alpha': 0.0001}
Best score : 0.77


### Save model

In [5]:
import joblib

joblib.dump(grid.best_estimator_, "../data/lasso_regression.pkl")

['../data/lasso_regression.pkl']